In [1]:

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge

Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


In [60]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Brooklyn").text

In [61]:
soup = BeautifulSoup(data, 'html.parser')

In [62]:
neighborhoodList = []

In [63]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [76]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)"
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)"
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)"
3,"► Boerum Hill‎ (1 C, 15 P)"
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)"


In [77]:
# print the number of rows of the dataframe
kl_df.shape


(40, 1)

In [79]:
! pip install geocoder
import geocoder
print("Success!")

     |████████████████████████████████| 102kB 7.5MB/s ta 0:00:011
Success!


In [80]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, New York City, New York'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [83]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [84]:
coords

[[40.63391000000007, -74.01490999999999],
 [40.68339000000003, -73.94246999999996],
 [40.601860000000045, -73.99389999999994],
 [40.68372000000005, -73.98178999999999],
 [40.63882000000007, -73.98911999999996],
 [40.576377578033984, -73.96818667069948],
 [40.695350000000076, -73.99404999999996],
 [40.67878471436522, -73.94408368906659],
 [40.662900000000036, -73.91728999999998],
 [40.71348811081892, -73.94145391606621],
 [40.63232473697458, -73.89237243313765],
 [40.68661000000003, -73.96753999999999],
 [40.698655631723405, -73.9697375878223],
 [40.68978000000004, -73.99816999999996],
 [40.65024396130458, -73.9720262791916],
 [40.66510012016697, -73.94815400957425],
 [40.69036389584243, -73.87739375767887],
 [40.69363000000004, -73.98287999999997],
 [40.62133000000006, -74.00993999999997],
 [40.65379000000007, -73.93080999999995],
 [40.66643000000005, -73.88268999999997],
 [40.67075814494184, -73.96774507356284],
 [40.62435102205693, -73.93053430948972],
 [40.69319000000007, -73.971679

In [85]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [86]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [87]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(40, 3)


,Neighborhood,Latitude,Longitude
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.633910,-74.014910
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",40.683390,-73.942470
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",40.601860,-73.993900
3,"► Boerum Hill‎ (1 C, 15 P)",40.683720,-73.981790
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)",40.638820,-73.989120
5,"► Brighton Beach‎ (1 C, 15 P)",40.576378,-73.968187
6,"► Brooklyn Heights‎ (3 C, 40 P)",40.695350,-73.994050
7,► Brooklyn Navy Yard‎ (12 P),40.678785,-73.944084
8,"► Brownsville, Brooklyn‎ (1 C, 12 P)",40.662900,-73.917290
9,"► Bushwick, Brooklyn‎ (1 C, 37 P)",40.713488,-73.941454


In [88]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

In [90]:
# get the coordinates of Brooklyn
address = 'Brooklyn, New York'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn, New York {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn, New York 40.6501038, -73.9495823.


In [91]:

map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [92]:
# save the map as HTML file
map_kl.save('map_kl.html')

In [93]:
# define Foursquare Credentials and Version
CLIENT_ID = 'O40DHTXH00T40LUTYMBQ3DPTEZ0VW0P3E34V30IPWYVMQV1L' # your Foursquare ID
CLIENT_SECRET = 'Z0VDZD5Y5D0MNJJR5LCGJU3KKPCRAVCJJMWLRATZAE0ZY4QA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O40DHTXH00T40LUTYMBQ3DPTEZ0VW0P3E34V30IPWYVMQV1L
CLIENT_SECRET:Z0VDZD5Y5D0MNJJR5LCGJU3KKPCRAVCJJMWLRATZAE0ZY4QA


In [94]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [95]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3953, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.63391,-74.01491,Prince Tea House 王室茶園,40.635882,-74.012841,Tea Room
1,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.63391,-74.01491,Leif Ericson Tennis Courts,40.632293,-74.013868,Tennis Court
2,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.63391,-74.01491,East Harbor Seafood Palace (迎賓大酒樓),40.633526,-74.014372,Seafood Restaurant
3,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.63391,-74.01491,Snow & Cream,40.635932,-74.012795,Dessert Shop
4,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",40.63391,-74.01491,Thanh Da,40.636588,-74.012029,Vietnamese Restaurant


In [96]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",100,100,100,100,100,100
"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",100,100,100,100,100,100
"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",100,100,100,100,100,100
"► Boerum Hill‎ (1 C, 15 P)",100,100,100,100,100,100
"► Borough Park, Brooklyn‎ (1 C, 18 P)",100,100,100,100,100,100
"► Brighton Beach‎ (1 C, 15 P)",100,100,100,100,100,100
"► Brooklyn Heights‎ (3 C, 40 P)",100,100,100,100,100,100
► Brooklyn Navy Yard‎ (12 P),100,100,100,100,100,100
"► Brownsville, Brooklyn‎ (1 C, 12 P)",100,100,100,100,100,100


In [97]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 284 uniques categories.


In [98]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Tea Room', 'Tennis Court', 'Seafood Restaurant', 'Dessert Shop',
       'Vietnamese Restaurant', 'Market', 'Wine Bar', 'Pizza Place',
       'Deli / Bodega', 'Italian Restaurant', 'Bakery', 'Bubble Tea Shop',
       'Dim Sum Restaurant', 'Gym / Fitness Center', 'Restaurant', 'Spa',
       'Greek Restaurant', 'Pet Store', 'American Restaurant',
       'Cantonese Restaurant', 'Coffee Shop', 'Turkish Restaurant',
       'Hotpot Restaurant', 'Lebanese Restaurant', 'Indian Restaurant',
       'Bar', 'Bagel Shop', 'Park', 'Scenic Lookout',
       'Mexican Restaurant', 'Burger Joint', 'Chinese Restaurant',
       'Diner', 'Asian Restaurant', 'Cosmetics Shop', 'Flower Shop',
       'Dumpling Restaurant', 'Café', 'Hookah Bar',
       'Vegetarian / Vegan Restaurant', 'Middle Eastern Restaurant',
       'Brewery', 'Cheese Shop', 'Szechuan Restaurant', 'Sports Bar',
       'Garden', 'Breakfast Spot', 'Taco Place', 'Food & Drink Shop',
       'Dog Run'], dtype=object)

In [99]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [100]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(3953, 285)


,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool Hall,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Street Art,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wareh

In [101]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(40, 285)


,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Island,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool Hall,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,Steakhouse,Street Art,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Varenyky restaurant,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wareh

In [105]:
len(kl_grouped[kl_grouped["Museum"] > 0])

8

In [109]:
kl_museum = kl_grouped[["Neighborhoods","Museum"]]

In [110]:
kl_museum.head()

,Neighborhoods,Museum
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.00
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",0.01
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",0.00
3,"► Boerum Hill‎ (1 C, 15 P)",0.00
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)",0.00


In [111]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 0, 0, 2, 0, 1, 0, 0], dtype=int32)

In [112]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_museum.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [113]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Museum,Cluster Labels
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.00,0
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",0.01,2
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",0.00,0
3,"► Boerum Hill‎ (1 C, 15 P)",0.00,0
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)",0.00,0


In [114]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(40, 5)


,Neighborhood,Museum,Cluster Labels,Latitude,Longitude
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.00,0,40.63391,-74.01491
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",0.01,2,40.68339,-73.94247
2,"► Bensonhurst, Brooklyn‎ (1 C, 28 P)",0.00,0,40.60186,-73.99390
3,"► Boerum Hill‎ (1 C, 15 P)",0.00,0,40.68372,-73.98179
4,"► Borough Park, Brooklyn‎ (1 C, 18 P)",0.00,0,40.63882,-73.98912


In [115]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(40, 5)


,Neighborhood,Museum,Cluster Labels,Latitude,Longitude
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.00,0,40.633910,-74.014910
20,"► East New York, Brooklyn‎ (2 C, 35 P)",0.00,0,40.666430,-73.882690
21,"► Flatbush, Brooklyn‎ (2 C, 43 P)",0.00,0,40.670758,-73.967745
22,"► Flatlands, Brooklyn‎ (1 C, 5 P)",0.00,0,40.624351,-73.930534
23,"► Fort Greene, Brooklyn‎ (1 C, 28 P)",0.00,0,40.693190,-73.971680
24,"► Gowanus, Brooklyn‎ (11 P)",0.00,0,40.672360,-73.990270
26,"► Greenpoint, Brooklyn‎ (1 C, 26 P)",0.00,0,40.730603,-73.950032
27,"► Manhattan Beach, Brooklyn‎ (1 C, 9 P)",0.00,0,40.577370,-73.947740
28,"► Marine Park, Brooklyn‎ (1 C, 9 P)",0.00,0,40.604130,-73.931150
29,"► Midwood, Brooklyn‎ (2 C, 20 P)",0.00,0,40.623260,-73.956530


In [116]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [117]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [118]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Museum,Cluster Labels,Latitude,Longitude
0,"► Bay Ridge, Brooklyn‎ (1 C, 18 P)",0.0,0,40.633910,-74.014910
20,"► East New York, Brooklyn‎ (2 C, 35 P)",0.0,0,40.666430,-73.882690
21,"► Flatbush, Brooklyn‎ (2 C, 43 P)",0.0,0,40.670758,-73.967745
22,"► Flatlands, Brooklyn‎ (1 C, 5 P)",0.0,0,40.624351,-73.930534
23,"► Fort Greene, Brooklyn‎ (1 C, 28 P)",0.0,0,40.693190,-73.971680
24,"► Gowanus, Brooklyn‎ (11 P)",0.0,0,40.672360,-73.990270
26,"► Greenpoint, Brooklyn‎ (1 C, 26 P)",0.0,0,40.730603,-73.950032
27,"► Manhattan Beach, Brooklyn‎ (1 C, 9 P)",0.0,0,40.577370,-73.947740
28,"► Marine Park, Brooklyn‎ (1 C, 9 P)",0.0,0,40.604130,-73.931150
29,"► Midwood, Brooklyn‎ (2 C, 20 P)",0.0,0,40.623260,-73.956530


In [119]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Museum,Cluster Labels,Latitude,Longitude
7,► Brooklyn Navy Yard‎ (12 P),0.02,1,40.678785,-73.944084
15,"► Crown Heights, Brooklyn‎ (4 C, 33 P)",0.03,1,40.665100,-73.948154
32,► People from Brooklyn by neighborhood‎ (35 C),0.02,1,40.678785,-73.944084
19,"► East Flatbush, Brooklyn‎ (10 P)",0.02,1,40.653790,-73.930810


In [120]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Museum,Cluster Labels,Latitude,Longitude
5,"► Brighton Beach‎ (1 C, 15 P)",0.01,2,40.576378,-73.968187
34,"► Red Hook, Brooklyn‎ (1 C, 21 P)",0.01,2,40.675020,-74.010230
25,"► Gravesend, Brooklyn‎ (1 C, 4 P)",0.01,2,40.586280,-73.998051
1,"► Bedford–Stuyvesant, Brooklyn‎ (1 C, 46 P)",0.01,2,40.683390,-73.942470
